## **FP Growth Algorithm**  
### **Author:** Hansal Shah 

# **Initializing the dataset**

In [ ]:
transactions = {'T100':['I1','I2','I5'], 'T200':['I2','I4'], 'T300':['I2','I3'], 
                'T400':['I1','I2','I4'], 'T500':['I1','I3'], 'T600':['I2','I3'], 
                'T700':['I1','I3'], 'T800':['I1','I2','I3','I5'], 
                'T900':['I1','I2','I3']}

# finding unique items
unique = []

for transaction in transactions:
  for item in transactions[transaction]:
    unique.append(item)

unique = list(set(unique))
unique.sort()
unique

['I1', 'I2', 'I3', 'I4', 'I5']

# **Helper Functions and Classes**

In [ ]:
class Node:
  def __init__ (self, item, frequency, parent):
    self.item = item
    self.frequency = frequency
    self.parent = None
    self.children = {}
  
  def increment_frequency(self):
    self.frequency+=1

  def display_tree(self, ind=1):
    print ('  '*ind, self.item, ' ', self.frequency)
    for child in self.children.values():
        child.display_tree(ind+1)

def count_frequency(item, transactions):
  count = 0
  for transaction in transactions:
    if item in transactions[transaction]:
      count+=1
  return count

def buildTree(root, transactions):
  for transaction in list(transactions.values()):
    temp_node = root
    for item in transaction:
      if item in temp_node.children:
        temp_node = temp_node.children[item]
        temp_node.increment_frequency()
      else:
        temp_node.children[item] = Node(item, 1, temp_node)
        temp_node = temp_node.children[item]

# **Preparing the transactions for building the tree**

### **Taking the input of minimum support count**

In [ ]:
# Taking the minimum support count from the user

min_sup = int(input("Enter the value of minimum support: "))

Enter the value of minimum support: 2


### **Removing all the items with support count less than that of minimum count**

In [ ]:
# Finding the frequecny of each unique item
item_freq = {}
for item in unique:
  item_freq[item] = count_frequency(item, transactions)

# Removing the items with frequency lower than that of minimum support count
items_to_remove = []
for item in item_freq:
  if item_freq[item]<min_sup:
    items_to_remove.append(item)

for item in items_to_remove:
  del item_freq[item]

item_freq

{'I1': 6, 'I2': 7, 'I3': 6, 'I4': 2, 'I5': 2}

### **Arranging the items of transactions in order of their decreasing count**

In [ ]:
#sorting the items in descending order of their frequency
unique = list(dict(sorted(item_freq.items(), key=lambda kv: kv[1], 
                          reverse=True)).keys())
unique

['I2', 'I1', 'I3', 'I4', 'I5']

In [ ]:
#arranging the items in each transaction in decreasing order of their frequency
for transaction in transactions:
  transactions[transaction].sort(key=lambda x: unique.index(x))

transactions

{'T100': ['I2', 'I1', 'I5'],
 'T200': ['I2', 'I4'],
 'T300': ['I2', 'I3'],
 'T400': ['I2', 'I1', 'I4'],
 'T500': ['I1', 'I3'],
 'T600': ['I2', 'I3'],
 'T700': ['I1', 'I3'],
 'T800': ['I2', 'I1', 'I3', 'I5'],
 'T900': ['I2', 'I1', 'I3']}

# **Generating the FP Tree**

In [ ]:
#Generating a tree
root = Node(None, None, None)
buildTree(root, transactions)

In [ ]:
#Displaying the tree
root.display_tree()

   None   None
     I2   7
       I1   4
         I5   1
         I4   1
         I3   2
           I5   1
       I4   1
       I3   2
     I1   2
       I3   2


# **Generating the Conditional Base Pattern for each item**

### **Helper Functions**

In [ ]:
# Generating the conditional pattern base

conditional_pattern = {}
frequencies = {}

def generateConditionalBasePattern(root, item, path=[]):
  if root.item==item:
    if len(path)!=0:
      if item in conditional_pattern:
        conditional_pattern[item].append(path.copy())
      else:
        conditional_pattern[item] = [path.copy()]

      if item in frequencies:
        frequencies[item].append(root.frequency)
      else:
        frequencies[item] = [root.frequency]

  else:
    for child in root.children:
      if root.item!=None:
        path.append(root.item)
        generateConditionalBasePattern(root.children[child], item, path)
        path.pop()
      else:
        generateConditionalBasePattern(root.children[child], item, path)

### **Generating and printing conditional base patterns**

In [ ]:
#Forming the conditional base pattern for each item starting from the item with 
#least support count


print('The minimum support value is: {}'.format(min_sup))
print(end="\n")

print("Conditional Base Patterns\n")
for i in range(len(unique)-1, -1, -1):
  generateConditionalBasePattern(root, unique[i])

for item in conditional_pattern:
  if len(conditional_pattern[item])!=0:
    print(item+'  '+'{', end="")
    for j in range(len(conditional_pattern[item])):
      if len(conditional_pattern[item][j])!=0:
        print(conditional_pattern[item][j], end="")
        if j!=len(conditional_pattern[item])-1:
          print(': '+str(frequencies[item][j])+', ',end="")
        else:
          print(': '+str(frequencies[item][j]), end="")
    print('}')

The minimum support value is: 2

Conditional Base Patterns

I5  {['I2', 'I1']: 1, ['I2', 'I1', 'I3']: 1}
I4  {['I2', 'I1']: 1, ['I2']: 1}
I3  {['I2', 'I1']: 2, ['I2']: 2, ['I1']: 2}
I1  {['I2']: 4}


# **Generating conditional FP tree**

### **Helper Functions**

In [ ]:
c_fp_tree = {}

def conditional_fp_tree(conditional_pattern, frequencies, ordered_items):
  for i in range(len(ordered_items)-1,-1,-1):
    c_fp_tree[ordered_items[i]] = []

    for item in ordered_items:
      if ordered_items[i] in conditional_pattern:
        temp_dict = {}
        for j in range(len(conditional_pattern[ordered_items[i]])):
          if len(conditional_pattern[ordered_items[i]][j])==0:
            continue
          else:
            if conditional_pattern[ordered_items[i]][j][0]==item:

              for each in conditional_pattern[ordered_items[i]][j]:
                if each not in temp_dict:
                  temp_dict[each] = frequencies[ordered_items[i]][j]
                else:
                  temp_dict[each] += frequencies[ordered_items[i]][j]

        items_in_dict = list(temp_dict.keys()).copy()
        for each in items_in_dict:
          if temp_dict[each]<min_sup:
            del temp_dict[each]

        if len(temp_dict)!=0:
          c_fp_tree[ordered_items[i]].append(temp_dict.copy())    

### **Generating conditional FP tree**

In [ ]:
conditional_fp_tree(conditional_pattern, frequencies, unique)

### **Printing the Conditional FP Tree**

In [ ]:
print("Conditional FP Tree\n")
for i in range(len(unique)-1, -1, -1):
  if len(c_fp_tree[unique[i]])!=0:
    print(unique[i], end='   ')
    print(c_fp_tree[unique[i]],end="\n")

Conditional FP Tree

I5   [{'I2': 2, 'I1': 2}]
I4   [{'I2': 2}]
I3   [{'I2': 4, 'I1': 2}, {'I1': 2}]
I1   [{'I2': 4}]


In [ ]:
c_fp_tree #to be removed

{'I1': [{'I2': 4}],
 'I2': [],
 'I3': [{'I1': 2, 'I2': 4}, {'I1': 2}],
 'I4': [{'I2': 2}],
 'I5': [{'I1': 2, 'I2': 2}]}

# **Generating Frequent Patterns**

### **Helper Functions**

In [ ]:
fpg_size_wise = {}
frequent_patterns = {}
pattern_frequencies = {}

def find_frequent_patterns(subset_item, subset_frequencies, 
                           itemset, frequency, index, item):

  if index==len(itemset): 
    if len(subset_item)!=0:
      subset_item.append(item)
      if item in frequent_patterns:

        if subset_item in frequent_patterns[item]:
          pattern_frequencies[item][frequent_patterns[item].index(subset_item)]+=min(subset_frequencies.copy())  
        else:
          frequent_patterns[item].append(subset_item.copy())
          pattern_frequencies[item].append(min(subset_frequencies.copy()))
      
      else:
        frequent_patterns[item] = [subset_item.copy()]
        pattern_frequencies[item] = [min(subset_frequencies.copy())]

      if len(subset_item) in fpg_size_wise:
        if subset_item not in fpg_size_wise[len(subset_item)]:
          fpg_size_wise[len(subset_item)].append(subset_item.copy())
      else:
        fpg_size_wise[len(subset_item)] = [subset_item.copy()]

      subset_item.pop()
    return

  #excluding the element
  find_frequent_patterns(subset_item, subset_frequencies, 
                         itemset, frequency, index+1, item)

  #including the element
  subset_item.append(itemset[index])
  subset_frequencies.append(frequency[index])
  find_frequent_patterns(subset_item, subset_frequencies, 
                         itemset, frequency, index+1, item)
  subset_item.pop()
  subset_frequencies.pop()

def generate_frequent_pattenrs(conditional_fp_tree):
  for item in conditional_fp_tree:
    for each in conditional_fp_tree[item]:
      find_frequent_patterns([],[],list(each.keys()),list(each.values()),0,item)

### **Generating Frequent Patterns**

In [ ]:
generate_frequent_pattenrs(c_fp_tree)

In [ ]:
fpg_size_wise #to be removed

{2: [['I1', 'I5'],
  ['I2', 'I5'],
  ['I2', 'I4'],
  ['I1', 'I3'],
  ['I2', 'I3'],
  ['I2', 'I1']],
 3: [['I2', 'I1', 'I5'], ['I2', 'I1', 'I3']]}

In [ ]:
frequent_patterns #to be removed

{'I1': [['I2', 'I1']],
 'I3': [['I1', 'I3'], ['I2', 'I3'], ['I2', 'I1', 'I3']],
 'I4': [['I2', 'I4']],
 'I5': [['I1', 'I5'], ['I2', 'I5'], ['I2', 'I1', 'I5']]}

In [ ]:
pattern_frequencies #to be removed

{'I1': [4], 'I3': [4, 4, 2], 'I4': [2], 'I5': [2, 2, 2]}

### **Printing the frequent pattern along with their support count**

In [ ]:
print("Frequent Patterns Generated\n")
for i in range(len(unique)-1, -1,-1):
  if unique[i] in frequent_patterns:
    print(unique[i]+'   '+'{',end='')
    for j in range(len(frequent_patterns[unique[i]])):
      print(frequent_patterns[unique[i]][j],end='')
      print(': ',end='')
      if j!=len(frequent_patterns[unique[i]])-1:
        print(pattern_frequencies[unique[i]][j],end=', ')
      else:
        print(pattern_frequencies[unique[i]][j],end='')
    print('}',end='\n')

Frequent Patterns Generated

I5   {['I1', 'I5']: 2, ['I2', 'I5']: 2, ['I2', 'I1', 'I5']: 2}
I4   {['I2', 'I4']: 2}
I3   {['I1', 'I3']: 4, ['I2', 'I3']: 4, ['I2', 'I1', 'I3']: 2}
I1   {['I2', 'I1']: 4}


# **Generating Association Rules**

### **Helper Functions**

In [ ]:
powerset = []

def generate_powerset(subset, index, itemset):

  if(index==len(itemset)):
    powerset.append(subset.copy())
    return

  #excluding the element
  generate_powerset(subset, index+1, itemset)

  #including the element
  subset.append(itemset[index])
  generate_powerset(subset, index+1, itemset)
  subset.pop()

def count_frequency_itemset(itemset, transactions):
  count = 0
  for transaction in transactions:
    if set(itemset).issubset(set(transactions[transaction])):
      count+=1
  return count

def generate_association_rules(l):
  possible_association_rules = {}
  final_association_rules = {}
  for i in range(2,max(list(l.keys()))+1):
    for itemset in l[i]:
      powerset.clear()
      generate_powerset([], 0, itemset)
      powerset.pop(0) 
      powerset.pop(-1)

      for s in powerset:
        # print(s)
        itemset_s = list(set(itemset)-set(s))
        count_s = count_frequency_itemset(s, transactions)
        count_itemset = count_frequency_itemset(itemset, transactions)

        rule = '{}=>{}'.format(set(s),set(itemset_s))
        confidence = (count_itemset/count_s)

        possible_association_rules[rule]=confidence

        if confidence>=min_conf:
          final_association_rules[rule] = confidence

  return possible_association_rules, final_association_rules

def print_rules_values(frequent_itemsets, confidence_values):
  print('Rule  |  Confidence')
  for i in range(len(frequent_itemsets)):
    print('{}   |   {}'.format(frequent_itemsets[i], confidence_values[i]))
  print('\n')

def print_rules_percentage(frequent_itemsets, confidence_values):
  print('Rule  |  Confidence(%)')
  for i in range(len(frequent_itemsets)):
    print('{}   |   {}'.format(frequent_itemsets[i], confidence_values[i]*100))
  print('\n')

### **Initializing the minimum confidence**

In [ ]:
min_conf = float(input('Enter the value of minimum confidence: '))

Enter the value of minimum confidence: 0.6


### **Generating the association rules**

In [ ]:
possible_association_rules, final_association_rules = generate_association_rules(fpg_size_wise)

### **Printing the minimum confidence and support**

In [ ]:
print("The value of minimum confidence is: {}".format(min_conf))
print("The value of minimum support is: {}".format(min_sup))

The value of minimum confidence is: 0.6
The value of minimum support is: 2


### **Printing the tentative set of rules and their confidence**

In [ ]:
print_rules_values(list(possible_association_rules.keys()), list(possible_association_rules.values()))

Rule  |  Confidence
{'I5'}=>{'I1'}   |   1.0
{'I1'}=>{'I5'}   |   0.3333333333333333
{'I5'}=>{'I2'}   |   1.0
{'I2'}=>{'I5'}   |   0.2857142857142857
{'I4'}=>{'I2'}   |   1.0
{'I2'}=>{'I4'}   |   0.2857142857142857
{'I3'}=>{'I1'}   |   0.6666666666666666
{'I1'}=>{'I3'}   |   0.6666666666666666
{'I3'}=>{'I2'}   |   0.6666666666666666
{'I2'}=>{'I3'}   |   0.5714285714285714
{'I1'}=>{'I2'}   |   0.6666666666666666
{'I2'}=>{'I1'}   |   0.5714285714285714
{'I5'}=>{'I2', 'I1'}   |   1.0
{'I1'}=>{'I5', 'I2'}   |   0.3333333333333333
{'I5', 'I1'}=>{'I2'}   |   1.0
{'I2'}=>{'I5', 'I1'}   |   0.2857142857142857
{'I5', 'I2'}=>{'I1'}   |   1.0
{'I2', 'I1'}=>{'I5'}   |   0.5
{'I3'}=>{'I2', 'I1'}   |   0.3333333333333333
{'I1'}=>{'I3', 'I2'}   |   0.3333333333333333
{'I3', 'I1'}=>{'I2'}   |   0.5
{'I2'}=>{'I3', 'I1'}   |   0.2857142857142857
{'I3', 'I2'}=>{'I1'}   |   0.5
{'I2', 'I1'}=>{'I3'}   |   0.5




### **Printing the final set of rules and their confidence (%)**

In [ ]:
print_rules_percentage(list(final_association_rules.keys()), list(final_association_rules.values()))

Rule  |  Confidence(%)
{'I5'}=>{'I1'}   |   100.0
{'I5'}=>{'I2'}   |   100.0
{'I4'}=>{'I2'}   |   100.0
{'I3'}=>{'I1'}   |   66.66666666666666
{'I1'}=>{'I3'}   |   66.66666666666666
{'I3'}=>{'I2'}   |   66.66666666666666
{'I1'}=>{'I2'}   |   66.66666666666666
{'I5'}=>{'I2', 'I1'}   |   100.0
{'I5', 'I1'}=>{'I2'}   |   100.0
{'I5', 'I2'}=>{'I1'}   |   100.0


